In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import kendalltau
import seaborn as sns

In [ ]:
a = pd.read_csv('/home/pecoffet/remoterobo/logs/negociate6/negociate-fit-lotrep-faketrue2-mutCoop0.01-uni0.01-2019-08-23-1805/donotkill_false/run_01/rep00/longrep1/fitnesslog.txt', delimiter="\t")
b = pd.read_csv('/home/pecoffet/remoterobo/logs/negociate6/negociate-fit-lotrep-faketrue2-mutCoop0.01-uni0.01-2019-08-23-1805/donotkill_false/run_01/rep00/longrep2/fitnesslog.txt', delimiter="\t")

In [ ]:
tota = a.groupby('ind', as_index=False).expanding().mean().reset_index()
totb = b.groupby('ind', as_index=False).expanding().mean().reset_index()

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(x='rep', y='fitness', hue='ind', data=totb.query('ind % 20 == 0 or ind == 65'))

In [ ]:
lastgena = tota.query(f'rep == {tota["rep"].max()}').copy()
lastgena['rank'] = lastgena.rank()['fitness']
lastgena['sim'] = 'a'

lastgenb = totb.query(f'rep == {totb["rep"].max()}').copy()
lastgenb['rank'] = lastgenb.rank()['fitness']
lastgenb['sim'] = 'b'

In [ ]:
lastgentot = pd.concat((lastgena, lastgenb))

In [ ]:
plt.figure(figsize=(4, 10))
sns.lineplot(x='sim', y='rank', hue='ind', data=lastgentot)

In [ ]:
kendalltau(lastgenb['rank'], lastgena['rank']).correlation

In [ ]:
%matplotlib inline

import tqdm

maxeval = min(int(tota['rep'].max()), int(totb['rep'].max()))

points = np.arange(0, maxeval, 100)
kendalls = []


for i in tqdm.tqdm_notebook(points):
    curgena = tota.query(f'rep == {i}').copy()
    curgena['rank'] = curgena.rank()['fitness']
    curgenb = totb.query(f'rep == {i}').copy()
    curgenb['rank'] = curgenb.rank()['fitness']
    kendallres = kendalltau(curgena['rank'], curgenb['rank'])
    kendalls.append({'rep': i, 'kendall': kendallres.correlation, 'pval': kendallres.pvalue })
kendalldf = pd.DataFrame(kendalls)
sorkendalldf = kendalldf.sort_values('rep')


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.set_ylim(0, 1)
ax.set_xlim(0, maxeval)
sorkendalldf.plot(x='rep', y='kendall', logx=True, ax=ax)

plt.show()

In [ ]:
kendalldf.plot(logy=True, x='rep', y='pval')